In [57]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#!pip install folium
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim as gyNominatim
import folium
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import ipywidgets as widgets
print('Libraries imported.')

Libraries imported.


# Part 1: Building the Neighborhoods Data Frame

In [58]:
#Get Data and populate data frame
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs1 = pd.read_html(url, attrs={'class': 'wikitable'})
Neighborhoods_df = pd.DataFrame(dfs1[0])

In [59]:
#Ignore cells with a borough that is Not assigned
Neighborhoods_df = Neighborhoods_df[Neighborhoods_df.Borough != 'Not assigned'].reset_index(drop=True)

In [60]:
#Combined neighborhoods with same postal code into one row separated with a comma
Neighborhoods_df['Neighborhood'] = Neighborhoods_df['Neighborhood'].str.replace(' /', ',')

In [61]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
Neighborhoods_df['Neighborhood'] = pd.np.where(Neighborhoods_df.Neighborhood.str.contains("Not assigned"), Neighborhoods_df.Borough, Neighborhoods_df.Neighborhood)

In [62]:
Neighborhoods_df.head(11)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [63]:
Neighborhoods_df.shape

(103, 3)

# Part 2: Add latitude and longitude to the Data Frame

In [64]:
#Get saved data for better latitude and longitude accuracy
lat_long = pd.read_csv('https://cocl.us/Geospatial_data')

In [65]:
#Add latitude and longitude to dataframe
Neighborhoods_df = Neighborhoods_df.join(lat_long.set_index('Postal Code'), on='Postal code')

In [66]:
#Build Geolocator for Canada
#geolocator = pgNominatim('CA')
#Get Latitude and Longitude
#Neighborhoods_df['Latitude'] = Neighborhoods_df['Postal code'].apply(geolocator.query_postal_code)[["latitude"]]
#Neighborhoods_df['Longitude'] = Neighborhoods_df['Postal code'].apply(geolocator.query_postal_code)[["longitude"]]

In [67]:
Neighborhoods_df.head(11)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Part 3: Neighborhoods Analysis and Clustering

## Analysis

In [68]:
#Drop Non Toronto Neighborhoods
toronto_neighborhoods_df = Neighborhoods_df[Neighborhoods_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [69]:
#Geolocator to get Toronto latitude and longitude
geogy = gyNominatim(user_agent="Toronto_map")

In [70]:
#Get Toronto latitude and longitude
location = geogy.geocode('Toronto, ON')

In [71]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods_df['Latitude'], toronto_neighborhoods_df['Longitude'], toronto_neighborhoods_df['Borough'], toronto_neighborhoods_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [72]:
# The code was removed by Watson Studio for sharing.

In [73]:
#Function to get the top 100 venues within a radius of 500 meters of a neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
        
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [74]:
#Get the top 100 venues within a radius of 500 meters of the neighborhoods
toronto_venues = getNearbyVenues(names=toronto_neighborhoods_df['Neighborhood'],
                                   latitudes=toronto_neighborhoods_df['Latitude'],
                                   longitudes=toronto_neighborhoods_df['Longitude']
                                  )

#Drop Neighborhood category
toronto_venues = toronto_venues[~toronto_venues['Venue Category'].str.contains('Neighborhood')].reset_index(drop=True)

In [75]:
#Resulting dataframe shape and sample
print(toronto_venues.shape)
toronto_venues.head()

(1689, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [76]:
#Number of venues per Neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,77,77,77,77,77,77
Christie,18,18,18,18,18,18
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [77]:
#Number of unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [78]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Neighborhood = toronto_venues['Neighborhood']
toronto_onehot.insert(0, 'Neighborhood', Neighborhood)

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
#Dataframe size
toronto_onehot.shape

(1689, 236)

In [80]:
#Calculating the mean of the frequency of occurrence of each category per Neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012987,0.000000,...,0.000000,0.000000,0.012987,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012346,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012346,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.012346,0.000000,0.024691
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [81]:
#New dataframe size
toronto_grouped.shape

(39, 236)

In [82]:
#Top 5 venues per Neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.04
3        Bakery  0.04
4    Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2       Nightclub  0.09
3     Coffee Shop  0.09
4    Intersection  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0           Yoga Studio  0.07
1         Auto Workshop  0.07
2         Garden Center  0.07
3    Light Rail Station  0.07
4  Fast Food Restaurant  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3               Plane  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1   Italian Restaurant  0.05
2         Burger Jo

In [83]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
#Create Dataframe with 10 most common venues per neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Farmers Market,Restaurant,Cheese Shop,Café,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Climbing Gym,Restaurant,Burrito Place
2,Business reply mail Processing CentrE,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Coffee Shop,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Burger Joint,Sandwich Place,Gym / Fitness Center,Spa,Middle Eastern Restaurant,Ice Cream Shop


## Clustering

In [91]:
#Function to cluster and visualize on map

def cluster_visualize(k):
    
    global toronto_merged
    # set number of clusters
    kclusters = k

    toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

    # check cluster labels generated for each row in the dataframe
    #kmeans.labels_[0:10] 
    
    #Create Dataframe with all data merged

    # add clustering labels
    neighborhoods_venues_sorted.drop("Cluster Labels", errors = "ignore", inplace=True, axis = 1)
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

    toronto_merged = toronto_neighborhoods_df

    # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
    toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

    #toronto_merged.head() # check the last columns!
    
    #Visualizing the clusters
    # create map
    map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
    return map_clusters

In [92]:
cluster_visualize(k = 5)

In [93]:
cluster_visualize(k = 7)

In [94]:
cluster_visualize(k = 10)

In [95]:
#Cluster 1 Analysis
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,0,Grocery Store,Café,Park,Gas Station,Italian Restaurant,Candy Store,Restaurant,Diner,Athletics & Sports,Baby Store
9,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Grocery Store,Fast Food Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pool,Recording Studio,Café
22,"High Park, The Junction South",0,Mexican Restaurant,Bar,Café,Thai Restaurant,Cajun / Creole Restaurant,Diner,Bakery,Italian Restaurant,Speakeasy,Flea Market


In [96]:
#Cluster 2 Analysis
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",1,Gym,Playground,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [97]:
#Cluster 3 Analysis
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",2,Coffee Shop,Pub,Park,Restaurant,Breakfast Spot,Theater,Café,Bakery,Mexican Restaurant,Chocolate Shop
1,"Queen's Park, Ontario Provincial Government",2,Coffee Shop,Yoga Studio,Juice Bar,Boutique,Café,Italian Restaurant,Beer Bar,Distribution Center,Discount Store,Bar
2,"Garden District, Ryerson",2,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Bookstore,Theater,Middle Eastern Restaurant,Restaurant
3,St. James Town,2,Coffee Shop,Café,Italian Restaurant,Restaurant,Breakfast Spot,Bakery,Diner,Cocktail Bar,American Restaurant,Beer Bar
5,Berczy Park,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Farmers Market,Restaurant,Cheese Shop,Café,Diner
6,Central Bay Street,2,Coffee Shop,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Burger Joint,Sandwich Place,Gym / Fitness Center,Spa,Middle Eastern Restaurant,Ice Cream Shop
8,"Richmond, Adelaide, King",2,Restaurant,Coffee Shop,Café,Bar,Bakery,Thai Restaurant,Office,Concert Hall,Cosmetics Shop,Juice Bar
10,"Harbourfront East, Union Station, Toronto Islands",2,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Sporting Goods Shop,Scenic Lookout,Brewery,Fried Chicken Joint,Restaurant
11,"Little Portugal, Trinity",2,Bar,Coffee Shop,Asian Restaurant,Restaurant,Café,Pizza Place,Men's Store,Vietnamese Restaurant,Italian Restaurant,New American Restaurant
12,"The Danforth West, Riverdale",2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Café,Spa,Brewery,Bubble Tea Shop
